In [38]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.cross_validation import KFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

In [39]:
data = pd.read_csv('data/new_features.csv', index_col='match_id')
print "Loaded"
datasets = {}
sc = StandardScaler()
for lt in [0, 1, 2]:
    print "Parsing LT = %d" % lt
    data_cur = data[data['lobby_type'] == lt]
    y = data_cur['radiant_win'].as_matrix()
    data_cur.drop(['radiant_win', 'lobby_type'], axis=1, inplace=True)
    data_cur = StandardScaler().fit_transform(data_cur)
    datasets[lt] = (data_cur, y)
    data = data[data['lobby_type'] != lt]
    print "Parsing LT = %d done" % lt

Loaded
Parsing LT = 0


/Library/Python/2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Parsing LT = 0 done
Parsing LT = 1
Parsing LT = 1 done
Parsing LT = 2
Parsing LT = 2 done


In [42]:
n_folds = 5
c = {
    0 : (0.001, 'l2'),
    1 : (0.00025, 'l2'),
    2 : (0.025, 'l1')
}
for lt in [2, 0, 1]:
    data, y = datasets[lt]
    cv = KFold(n=data.shape[0], n_folds=n_folds, shuffle=True)
    print "Training classifier for LT = {}, c = {}".format(lt, c[lt])
    auc_mean = 0.0
    for train_idx, test_idx in cv:
        C, penalty = c[lt]
        clf = LogisticRegression(C=C, penalty=penalty)
        clf.fit(data[train_idx], y[train_idx])
        y_pred = clf.predict_proba(data[test_idx])[:, 1]
        score = roc_auc_score(y[test_idx], y_pred)
        print score
        auc_mean += score
    auc_mean = auc_mean / float(n_folds)
    print "ROC AUC mean for LT = {}: {}".format(lt, auc_mean)

Training classifier for LT = 2, c = (0.025, 'l1')
0.72056069958
0.716448867963
0.722564288168
0.710761227908
0.707179820848
ROC AUC mean for LT = 2: 0.715502980894
Training classifier for LT = 0, c = (0.001, 'l2')
0.744008234402
0.760637400794
0.748654539508
0.730400703871
0.738433389758
ROC AUC mean for LT = 0: 0.744426853666
Training classifier for LT = 1, c = (0.00025, 'l2')
0.773353508995
0.776308095498
0.773902311689
0.773238846541
0.784049151221
ROC AUC mean for LT = 1: 0.776170382789
